# 基于多层耦合创新网络的关键核心技术识别

这个 Notebook 集成了网络数据处理与分析的多个执行步骤，包括数据收集、预处理、分析和可视化。

## 目录
1. [环境设置与导入库](#1-环境设置与导入库)
2. [数据收集](#2-数据收集)
3. [数据预处理](#3-数据预处理)
4. [网络分析](#4-网络分析)
5. [数据可视化](#5-数据可视化)
6. [结果导出](#6-结果导出)

## 1. 原始数据清洗

将所有英文字母转化为小写字母（防止漏掉重复值）,只保留“公开（公告）号、引文专利公开号、施引专利公开号、IPC分类、专利权人”五列数据；
删除无IPC号和乱码数据；
删除“公开（公告）号”列的重复值；
删除个人申请专利

In [7]:
# 1. 导入必要的库
import pandas as pd
from pathlib import Path

# 2. 设置数据路径（确保和你的文件结构一致）
DATA_DIR = Path("./data")

# 3. 从 step1 的两个文件导入函数
from algorithms.step_1_clean_patent_data import clean_patent_data
from algorithms.step_1_remove_personal_application import remove_personal_applications

# 4. 直接调用函数（无需修改原函数代码）
print("🚀 开始数据清洗...")
clean_patent_data()                # 执行第一步清洗
remove_personal_applications()     # 执行第二步清洗
print("✅ 数据清洗完成！")

🚀 开始数据清洗...
数据清洗失败：[Errno 2] No such file or directory: '..\\data\\input\\original_patent_data.xlsx'
数据处理异常：[Errno 2] No such file or directory: '..\\data\\step1_output\\patent_data_cleaned.xlsx'
✅ 数据清洗完成！


## 2. 多层耦合创新网络构建

这一步骤包括构建知识网络、技术网络、合作研发网络三个网络层和知识-技术网络、知识-合作研发网络、技术-合作研发网络三个层间网络

## 3. 网络层权重计算

使用多pagerank中心性算法计算出网络层中心性，得到网络层权重。

## 4. 节点关键性计算

节点在多层耦合创新网络中的结构洞

## 5. 节点核心性计算

节点在多层耦合创新网络中的中心度

## 6. 指标权重计算

读取三个网络层内节点名称及节点的关键性和核心性数据，构建关键性和核心性数据库

## 7.节点关键核心性计算

读取关键性和核心性数据库、指标权重，计算节点关键核心性

## 总结

这个 Notebook 提供了一个完整的网络数据处理与分析流程：

1. **环境设置**: 导入必要的库并配置环境
2. **数据收集**: 生成或加载网络数据
3. **数据预处理**: 清理和准备数据
4. **网络分析**: 计算各种网络指标
5. **数据可视化**: 创建图表展示分析结果
6. **结果导出**: 保存处理后的数据和分析结果

您可以根据具体需求修改和扩展这些步骤。